In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

data
map 1 = 0/1
map 2 = 4/5
map 3 = 6/7

In [73]:
def get_totals(cell):
    return cell.split()[0]

def get_totals_d(cell):
    return cell.split()[1]

def format_dataframe(dataframes, map_num):
    """
    unduplicates data

    :param dataframes: dataframes returned using the pandas read_html function on the page source
    :param map_num: indexed starting at 1 for which map is being played in a series
    :return: formatted df
    """

    if map_num == 1:
        formatting_df = pd.concat(dataframes[:2])
    elif map_num == 2:
        formatting_df = pd.concat([dataframes[4], dataframes[5]])
    elif map_num == 3:
        formatting_df = pd.concat([dataframes[6], dataframes[7]])


    formatting_df = formatting_df.reset_index()
    formatting_df = formatting_df.drop(columns=['index', 'Unnamed: 1'])

    formatting_df.iloc[:, 1:4] = formatting_df.iloc[:, 1:4].applymap(get_totals)
    formatting_df.iloc[:, 5:] = formatting_df.iloc[:, 5:].applymap(get_totals)
    formatting_df['PLAYER'] = formatting_df['Unnamed: 0'].apply(lambda x: x.split()[0])
    formatting_df['TEAM'] = formatting_df['Unnamed: 0'].apply(lambda x: x.split()[1])

    formatting_df = formatting_df.drop(columns=['Unnamed: 0'])

    formatting_df.iloc[:, 3] = formatting_df.iloc[:, 3].apply(get_totals_d)

    return formatting_df


def get_match_results_data(source, link):
    """
    :param source: pagesource
    :param link: link for particular match results page
    :return: multiple dataframes of match results, names for images/agent names
    """
    dfs = pd.read_html(source)

    img_elements = driver.find_elements(By.TAG_NAME, value='img')

    # Loop through each image element and extract the "alt" text
    alt_texts = []
    for img_element in img_elements:
        alt_text = img_element.get_attribute('alt')
        if alt_text:
            alt_texts.append(alt_text)


    return dfs, alt_texts

team information
0/1 = map 1 = 0
4/5 = map 2 = 1
8/9 = map 3 = 2


In [198]:
def get_data(driver, map_num, map_name):
    dfs = pd.read_html(driver.page_source)
    df1 = format_dataframe(dfs, map_num)


    #define element needed to be searched for team information depending on map number
    if map_num == 1:
        team_nums = [0,1]
    elif map_num == 2:
        team_nums = [4,5]
    elif map_num == 3:
        team_nums = [8,9]

    #add agents
    dfs, alt_texts = get_match_results_data(driver.page_source, links[0])
    df1['AGENT'] = alt_texts[2:12]

    # Get Time/Teams/Map for creating a primary key
    time_of_map = driver.find_element(By.CLASS_NAME, value='moment-tz-convert')
    time_of_map = time_of_map.get_attribute('data-utc-ts')

    # map_name = driver.find_element(By.CLASS_NAME, value='map')
    # print(map_name.text)
    # map_name = map_name.text.split('\n')[0]
    df1['map'] = map_name


    #get rounds won and team names
    team_information = driver.find_elements(By.CLASS_NAME, value='vm-stats-game-header')[map_num-1]
    right_team = team_information.find_element(By.CLASS_NAME, value='team.mod-right')
    left_team = team_information.find_element(By.CLASS_NAME, value='team')

    formatted_team_information = right_team.text.split('\n')

    away_team_information = formatted_team_information[0]
    away_rounds_won = formatted_team_information[-1]
    df1['rounds_won'] = 0
    df1['rounds_won'].iloc[5:] = away_rounds_won

    formatted_team_information_home = left_team.text.split('\n')

    home_team_information = formatted_team_information_home[-2]
    home_rounds_won = formatted_team_information_home[0]
    df1['rounds_won'].iloc[:5] = home_rounds_won

    #create primary key
    df1['match_id'] = home_team_information + ' ' + away_team_information + ' ' + map_name + ' ' + time_of_map

    return df1

In [75]:
matches_path = 'https://www.vlr.gg/matches/results'
chromedriver_path = r'C:\Users\work\Sportsbetting\Valorant\chromedriver.exe'

In [76]:
every_day_container_class = 'col.mod-1'
# create a new Chrome browser instance


### GET LINKS TO MATCH RESULTS

In [77]:
driver = webdriver.Chrome(executable_path=chromedriver_path)
# navigate to a website
driver.get(matches_path)

anchors = driver.find_elements(by=By.CSS_SELECTOR, value='a')

links = []
# Iterate over each anchor element and extract the href attribute
for anchor in anchors:
    href = anchor.get_attribute('href')
    links.append(href)

# container = driver.find_element(by=By.CLASS_NAME, value=every_day_container_class)
# nested_divs = container.find_elements(by=By.CSS_SELECTOR, value='div')
#
# data = []
# for div in nested_divs:
#     data.append((div.text, div.href))

C:\Users\work\AppData\Local\Temp\ipykernel_15692\3993812057.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chromedriver_path)


In [203]:
links = links[14:-10]

In [199]:
links[0]

'https://www.vlr.gg/197253/guild-x-vs-acend-rising-game-changers-2023-series-i-emea-groups-r2'

In [212]:
driver.get(links[3])

try:
    more_than_one_map = driver.find_element(By.CLASS_NAME, value='vm-stats-gamesnav.noselect  ')
except NoSuchElementException:
    print('only one map')
    more_than_one_map = None

IndexError: list index out of range

In [209]:
dataframes = []

if more_than_one_map:
    map_buttons_1 = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/div[3]/div[6]/div/div[1]/div[2]/div/div[2]')
    map_buttons_2 = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/div[3]/div[6]/div/div[1]/div[2]/div/div[3]')
    map_buttons_3 = driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/div[3]/div[6]/div/div[1]/div[2]/div/div[4]')
    print(map_buttons_1.text, map_buttons_2.text, map_buttons_3.text)
    map_buttons = [map_buttons_1, map_buttons_2, map_buttons_3]

    for map, button in enumerate(map_buttons):
        if 'N/A' not in button.text:
            button.click()
            df = get_data(driver, map+1, map_buttons[map].text.split()[1])
            dataframes.append(df)


else:
    df = get_data(driver, 1, driver.find_element(By.CLASS_NAME, value='map').text.split('\n')[0])
    dataframes.append(df)

1 Ascent 2 Haven 3 N/A


C:\Users\work\AppData\Local\Temp\ipykernel_15692\182681341.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won
C:\Users\work\AppData\Local\Temp\ipykernel_15692\182681341.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['rounds_won'].iloc[5:] = away_rounds_won


In [210]:
dataframes[0]

,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1,PLAYER,TEAM,AGENT,map,rounds_won,match_id
0,1.31,248,22,14,6,+8,77%,164,41%,3,2,+1,quin,ODG,omen,Ascent,14,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
1,1.16,192,15,15,15,0,65%,136,44%,1,2,-1,SiMon,ODG,kayo,Ascent,14,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
2,1.12,249,22,19,2,+3,54%,151,29%,5,5,0,TianYan,ODG,jett,Ascent,14,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
3,0.92,202,17,20,4,-3,54%,131,22%,2,2,0,razkiNgz,ODG,killjoy,Ascent,14,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
4,0.75,90,8,16,5,-8,58%,63,42%,1,3,-2,dxy,ODG,sova,Ascent,14,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
5,1.15,212,19,16,7,+3,77%,136,23%,3,1,+2,x8,KBG,sova,Ascent,12,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
6,1.12,198,16,15,15,+1,88%,135,14%,1,0,+1,Yiwaii,KBG,kayo,Ascent,12,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
7,0.98,207,19,17,6,+2,73%,136,16%,3,2,+1,KOKORO,KBG,killjoy,Ascent,12,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
8,0.95,179,15,17,11,-2,65%,109,22%,1,1,0,0n1y,KBG,omen,Ascent,12,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00
9,0.75,155,15,19,2,-4,62%,96,21%,6,8,-2,River,KBG,jett,Ascent,12,ODG KeepBest Gaming Ascent 2023-04-25 03:30:00


In [211]:
dataframes[1]

,R,ACS,K,D,A,+/–,KAST,ADR,HS%,FK,FD,+/–.1,PLAYER,TEAM,AGENT,map,rounds_won,match_id
0,1.44,260,21,14,10,+7,79%,164,25%,5,0,+5,quin,ODG,omen,Haven,13,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
1,1.18,271,23,17,2,+6,63%,183,32%,4,1,+3,razkiNgz,ODG,kayo,Haven,13,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
2,1.13,183,15,13,16,+2,83%,125,21%,0,0,0,SiMon,ODG,jett,Haven,13,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
3,0.89,187,16,17,3,-1,63%,126,44%,5,7,-2,TianYan,ODG,killjoy,Haven,13,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
4,0.89,133,13,14,3,-1,79%,90,60%,2,0,+2,dxy,ODG,sova,Haven,13,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
5,0.97,183,18,17,3,+1,63%,106,16%,1,1,0,Yiwaii,KBG,sova,Haven,11,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
6,0.97,191,15,18,5,-3,63%,119,26%,6,5,+1,River,KBG,kayo,Haven,11,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
7,0.97,156,13,16,7,-3,79%,103,26%,0,3,-3,0n1y,KBG,killjoy,Haven,11,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
8,0.85,194,17,19,6,-2,71%,118,18%,1,3,-2,KOKORO,KBG,omen,Haven,11,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
9,0.77,169,12,18,12,-6,75%,118,9%,0,4,-4,x8,KBG,jett,Haven,11,ODG KeepBest Gaming Haven 2023-04-25 03:30:00
